In [1]:
import numpy as np

In [2]:
def get_graph(text):
    f = open(text)
    info = []
    while True:
        line = f.readline()
        if line == '':
            break
        else:
            info.append(line)
    f.close()
    
    total_nodes = int(info[0].split('-')[1])
    start_node = int(info[1].split('-')[1])
    end_node = int(info[2].split('-')[1])
    
    graph_info = info[3:]
    
    graph_dict = {}
    for line in graph_info:
        l = line.split(',')
        nodes = l[0].split('-')
        node_1 = int(nodes[0])
        node_2 = int(nodes[1])
        distance = int(l[1])
        action = int(l[2])
        if node_1 in graph_dict.keys():
            graph_dict[node_1].append([distance, node_2,action])
        else:
            graph_dict[node_1] = []
            graph_dict[node_1].append([distance, node_2,action])
        if node_2 in graph_dict.keys():
            graph_dict[node_2].append([distance, node_1, -action])
        else:
            graph_dict[node_2] = []
            graph_dict[node_2].append([distance, node_1, -action])
    assert total_nodes == len(graph_dict.keys()), 'Fuck! Something is wrong'
    return start_node, end_node, graph_dict


    
    

In [3]:
def djikstra(maze_dict,start,goal, verbose = False):


    #maze_dict = graph_maze(maze)

    #start_position = (start[0],start[1])
    #goal_position = (goal[0],goal[1])


    open_list = []
    closed_list = []
    backpointer = {}
    path = []
    found = False
    resign = False

    x = start
    g = 0

    open_list.append((g,x))
    closed_list.append(x)
    steps = 0


    while not resign and not found:
        if verbose:
            print('Step {}'.format(steps))
            print('Open list:')
            print(open_list)
            print('Closed list')
            print(closed_list,end = '\n\n')
        if len(open_list)==0:
            print('It is impossible to solve this maze.')
            resign = True
        else:
            open_list.sort()
            current = open_list.pop(0)
            x = current[1]
            g = current[0]
            if x==goal:
                print('Goal reached in {} steps'.format(steps))
                found = True
                move_steps = 0
                path.append(x)
                while (x != start):
                    x = backpointer[x]
                    move_steps+=1
                    path.append(x)
                path.reverse()
                for node in path:
                    print('Node: {}'.format(node))
                print('Total cost: {}'.format(g))

            else:
                for neighbour in maze_dict[x]:
                    if neighbour[1] not in closed_list:
                        x2 = neighbour[1]
                        g2 = neighbour[0] + g
                        backpointer[x2] = x
                        open_list.append((g2,x2))
                        closed_list.append(x2)
        steps+=1
    print('Total time steps: {}'.format(steps))
    return path


In [4]:
start, end, maze_dict = get_graph('graph.txt')

In [5]:
start

52

In [6]:
end

53

In [7]:
maze_dict[start]

[[10, 51, 2]]

In [8]:
def get_pose(action, pose):
    assert action in [1,-1,2,-2], "Action non existent!"
    if action == 2:
        pose = np.array([[0],[-1]])
    elif action == 1:
        pose = np.array([[1],[0]])
    elif action == -1:
        pose = np.array([[-1],[0]])
    elif action == -2:
        pose = np.array([[0,1]])
    return pose
def get_rotation_matrix(pose):
    if (pose==np.array([[0],[1]])).all():
        rot_matrix = np.eye(2)
    elif (pose==np.array([[0],[-1]])).all():
        rot_matrix = -np.eye(2)
    elif (pose==np.array([[1],[0]])).all():
        rot_matrix = np.array([[0,-1],[1,0]])
    elif (pose==np.array([[-1],[0]])).all():
        rot_matrix = -np.array([[0,-1],[1,0]])
    return rot_matrix


In [9]:
path = djikstra(maze_dict,start,end)

Goal reached in 41 steps
Node: 52
Node: 51
Node: 48
Node: 46
Node: 17
Node: 14
Node: 44
Node: 43
Node: 12
Node: 7
Node: 42
Node: 53
Total cost: 192
Total time steps: 42


In [156]:
start

52

In [157]:
maze_dict[start]

[[10, 51, 2]]

In [10]:
def calculate_actions(current_position, pose, path):
    actions = []

    actions_matrice = np.array([[0,-2,0],[-1,0,1],[0,2,0]])
    for position in path:
        for i  in maze_dict[current_position]:
            print(i)
            if i[1]==position:
                action = i[2]
                assert action in [-1,1,-2,2]
        print(action)
        indexes = np.where(actions_matrice==action)
        print(indexes, type(indexes))
        indexes_np = np.array(indexes)
        print(pose, type(pose), pose.shape)
        indexes_np = indexes_np-1
        rot_matrix = get_rotation_matrix(pose)
        new_indexes = np.matmul(rot_matrix,indexes_np)
        new_indexes+=1
        print(new_indexes)
        new_action = actions_matrice[tuple(new_indexes.astype(int))]
        actions.append(new_action)
        print('new_action',new_action)
        pose = get_pose(action, pose)
        print('new_pose', pose)
        current_position  = position
    
    return actions
        
        




        

In [11]:
path

[52, 51, 48, 46, 17, 14, 44, 43, 12, 7, 42, 53]

In [12]:
start = path.pop(0)
init_pose = np.array([[0],[-1]])

In [13]:
calculate_actions(start, init_pose, path)

[10, 51, 2]
2
(array([2]), array([1])) <class 'tuple'>
[[ 0]
 [-1]] <class 'numpy.ndarray'> (2, 1)
[[0.]
 [1.]]
new_action [-2]
new_pose [[ 0]
 [-1]]
[12, 27, 1]
[29, 28, -1]
[28, 48, 2]
[10, 52, -2]
2
(array([2]), array([1])) <class 'tuple'>
[[ 0]
 [-1]] <class 'numpy.ndarray'> (2, 1)
[[0.]
 [1.]]
new_action [-2]
new_pose [[ 0]
 [-1]]
[12, 23, -1]
[14, 46, 2]
[28, 51, -2]
2
(array([2]), array([1])) <class 'tuple'>
[[ 0]
 [-1]] <class 'numpy.ndarray'> (2, 1)
[[0.]
 [1.]]
new_action [-2]
new_pose [[ 0]
 [-1]]
[12, 17, 2]
[12, 20, 1]
[16, 35, -1]
[14, 48, -2]
2
(array([2]), array([1])) <class 'tuple'>
[[ 0]
 [-1]] <class 'numpy.ndarray'> (2, 1)
[[0.]
 [1.]]
new_action [-2]
new_pose [[ 0]
 [-1]]
[13, 14, -1]
[12, 46, -2]
-1
(array([1]), array([0])) <class 'tuple'>
[[ 0]
 [-1]] <class 'numpy.ndarray'> (2, 1)
[[1.]
 [2.]]
new_action [1]
new_pose [[-1]
 [ 0]]
[13, 17, 1]
[15, 44, 2]
2
(array([2]), array([1])) <class 'tuple'>
[[-1]
 [ 0]] <class 'numpy.ndarray'> (2, 1)
[[1]
 [0]]
new_action [

[array([-2]),
 array([-2]),
 array([-2]),
 array([-2]),
 array([1]),
 array([-1]),
 array([-1]),
 array([1]),
 array([1]),
 array([-1]),
 array([-2])]

In [175]:
graph_dict[7]

[[29, 54, 2], [12, 58, -1]]

In [52]:
np.array([0.,3,4,5]).astype(int)

array([0, 3, 4, 5])